### AB Test in Python

`Experimentation` has become a non-negotiable factor in keeping up with the dynamic nature of customers, helping to understand what works or doesnt work. 

The goal of experimentation is `learning.` Learning about customers, market segments, price sensitivity, choice, impact of specific features, the list is endless. AB Test is an interesting way to achieve the learning goal of experimentation.

##### What is an AB Test?

An AB Test simply means comparing two or multiple versions of a test component (Which can be a new feature to your software program, modification to your website load-time speed, advertising copy, or evening your Call-to-Action button). If it impacts your KPIs then it should be tested.

An AB test always starts with an hypothesis. For example, you can propose that adding a new feature to your software program would increase the number of purchases that happen by users who visit your website. The truth is that you don't know if that holds true when the feature is eventually launched but that statement becomes your hypothesis, which you can then test with any experimentation method [An AB Test in this case].

For this short project, we go through a simple AB Test life-cyle for changes made to a webpage. We want to know if there is a significant change in conversion rate if we switch to a new webpage. Since we do not know the answer yet, we can run an AB Test to verify our hypothesis.

Our null hypothesis - Changing our webpage will increase our conversion rate by `3%`.

Out hypothesis simply states that if we had a conversion rate on `15%` with the old page, launching our new page will increase this conversion rate to `18%`

#### Project Outline
1. Data importation and Preparation
2. Defining our Experiment Criteria
    - Statistical Power
    - Minimum Detectible Effect (MDE)
    - alpha
    - Confidence Interval
    - Sample Size
3. Running the experiment
4. Testing our results
5. Decision Making


#### Data Importation and Preparation

In [7]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.stats.api as sms
import seaborn as sns
from math import ceil

%matplotlib inline
 

In [13]:
#Load Dataset
data = pd.read_csv(r"C:\Users\samue\OneDrive\Documents\Bobby\Datasets\AB Test\ab_data.csv")

In [14]:
data.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


#### Dataset Features
- user_id - The unique identifier for each session. Every user has a unique identifier
- timestamp - When the session happened.
- group - The group which the user was assigned to for that specific session {control, treatment}
- landing_page - Tells us if the user saw either the old page or the new page during their session.
- converted - Whether the session ended in a conversion or not (binary, 0=not converted, 1=converted)

In [16]:
pd.crosstab(data['group'], data['landing_page']) 

landing_page,new_page,old_page
group,,
control,1928,145274
treatment,145311,1965


In [18]:
#Let's check if any user has more than two sessions in our dataset
session_counts = data['user_id'].value_counts(ascending=False)
print(session_counts)
multi_users = session_counts[session_counts > 1].count()
print(f'There are {multi_users} users that appear multiple times in the dataset')

user_id
805339    2
754884    2
722274    2
783176    2
898232    2
         ..
642985    1
771499    1
923606    1
712675    1
715931    1
Name: count, Length: 290584, dtype: int64
There are 3894 users that appear multiple times in the dataset


In [20]:
#Drop users with multiple sessions so no user is sampled twice
users_to_drop = session_counts[session_counts > 1].index
data = data[~data['user_id'].isin(users_to_drop)]
print(f'The updated dataset now has {data.shape[0]} entries')

The updated dataset now has 286690 entries


In [2]:
#Calculating the effect size for out AB test experiment with the stats model package

effect_size = sms.proportion_effectsize(0.13,0.15)


In [11]:
#Calculating required sample size for our AB Test experiment
required_sample = sms.NormalIndPower().solve_power(
                                        effect_size,
                                        power = 0.8,
                                        alpha = 0.05,
                                        ratio = 1
                                                )

n = ceil(required_sample) #To the nearest whole number
print(f'We need {n} users for our experiment')

We need 4720 users for our experiment


In [21]:
control_sample= data[data['group'] == 'control'].sample(n=n, random_state=1)
treatment_sample = data[data['group'] == 'treatment'].sample(n=n, random_state=1)

In [22]:
ab_test_data = pd.concat([control_sample, treatment_sample], axis=0)
ab_test_data.reset_index(drop=True, inplace=True)

ab_test_data

,user_id,timestamp,group,landing_page,converted
0,788447,2017-01-15 10:15:53.966766,control,old_page,0
1,644367,2017-01-04 13:27:00.815306,control,old_page,0
2,921476,2017-01-13 11:28:38.186516,control,old_page,0
3,844813,2017-01-09 02:20:49.471715,control,old_page,0
4,675390,2017-01-09 23:51:06.765370,control,old_page,0
...,...,...,...,...,...
9435,716806,2017-01-07 01:57:26.210472,treatment,new_page,1
9436,779389,2017-01-13 20:23:58.696497,treatment,new_page,0
9437,679407,2017-01-04 21:38:48.113542,treatment,new_page,0
9438,930299,2017-01-06 11:01:43.273952,treatment,new_page,0


In [23]:
ab_test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9440 entries, 0 to 9439
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       9440 non-null   int64 
 1   timestamp     9440 non-null   object
 2   group         9440 non-null   object
 3   landing_page  9440 non-null   object
 4   converted     9440 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 368.9+ KB


In [24]:
ab_test_data['group'].value_counts()

group
control      4720
treatment    4720
Name: count, dtype: int64

In [42]:
conversion_rates = ab_test_data.groupby('group')['converted']
ab_test_data.groupby('group')['converted'].mean()


group
control      0.121610
treatment    0.112712
Name: converted, dtype: float64

In [43]:
#Calculate basic statistical properties of our experiment data
std_dev = lambda x: np.std(x, ddof=0)
std_error = lambda x: stats.sem(x, ddof=0)

conversion_rates = conversion_rates.agg([np.mean, std_dev, std_error])
conversion_rates.columns = {'comversion_rate', 'standard_deviation', 'standard_error'}
conversion_rates.style.format('{:.3f}')

,standard_error,comversion_rate,standard_deviation
group,,,
control,0.122,0.327,0.005
treatment,0.113,0.316,0.005


In [44]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [46]:
control_results = ab_test_data[ab_test_data['group'] == 'control']['converted']
treatment_results = ab_test_data[ab_test_data['group'] == 'treatment']['converted']

n_con = control_results.count()
n_treatment = treatment_results.count()

In [50]:
successes = [control_results.sum(), treatment_results.sum()]
nobs = [n_con, n_treatment]

z_stats, pval = proportions_ztest(successes, nobs=nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs, alpha=0.05)

print(f'z statistics: {z_stats:.2f}')
print(f'p-value: {pval:.3f}')
print(f'ci 95% for control group: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'ci 95% for treatment group: [{lower_treat:.3f}, {upper_treat:.3f}]')

z statistics: 1.34
p-value: 0.179
ci 95% for control group: [0.112, 0.131]
ci 95% for treatment group: [0.104, 0.122]


In [ ]:
# initialize all alpha variables, effect size variable and do an assert to check that they meet all conditions as a test
#put the data in a DB and pull it out with python
#T_test